In [224]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [225]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  ExtraTreesClassifier
import joblib

In [226]:
model_etc = joblib.load('/content/drive/MyDrive/Colab Notebooks/Projet 10/etc_faux_billets.joblib')
model_lrc = joblib.load('/content/drive/MyDrive/Colab Notebooks/Projet 10/lrc_faux_billets.joblib')

In [227]:
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projet 10/billets_production.csv', sep=',')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projet 10/Classeur1.csv', sep=';')

In [228]:
df

,Unnamed: 0,diagonal,height_left,height_right,margin_low,margin_up,length,id
0,0,172.09,103.95,103.73,4.39,3.09,113.19,B_1
1,1,171.52,104.17,104.03,5.27,3.16,111.82,B_2
2,2,171.78,103.80,103.75,3.81,3.24,113.39,B_3
3,3,172.02,104.08,103.99,5.57,3.30,111.10,B_4
4,4,171.79,104.34,104.37,5.00,3.07,111.87,B_5


In [229]:
df.set_index('id', drop=True, inplace=True)

In [230]:
df.drop(columns='Unnamed: 0', inplace=True)

In [231]:
df_test1=df.iloc[:,0:6]
predictions1 = model_etc.predict(df_test1)
proba_authenticite1 = model_etc.predict_proba(df_test1)
# Ajouter les prédictions au DataFrame
df_test1 = pd.DataFrame(df_test1)
df_test1['prédiction1'] = predictions1
df_authentique1 = df_test1[df_test1['prédiction1'] == 1]
df_non_authentique1 = df_test1[df_test1['prédiction1'] == 0]
df_test1['Proba authenticité1'] =  np.round((proba_authenticite1[:, 1])*100,2)
df_test1

,diagonal,height_left,height_right,margin_low,margin_up,length,prédiction1,Proba authenticité1
id,,,,,,,,
B_1,172.09,103.95,103.73,4.39,3.09,113.19,True,100.0
B_2,171.52,104.17,104.03,5.27,3.16,111.82,False,0.0
B_3,171.78,103.80,103.75,3.81,3.24,113.39,True,100.0
B_4,172.02,104.08,103.99,5.57,3.30,111.10,False,0.0
B_5,171.79,104.34,104.37,5.00,3.07,111.87,False,6.0


In [232]:
df_test2=df.iloc[:,0:6]
predictions = model_lrc.predict(df_test2)
proba_authenticite = model_lrc.predict_proba(df_test2)
# Ajouter les prédictions au DataFrame
df_test2 = pd.DataFrame(df_test2)
df_test2['prédiction2'] = predictions
df_authentique2 = df_test2[df_test2['prédiction2'] == 1]
df_non_authentique2 = df_test2[df_test2['prédiction2'] == 0]
df_test2['Proba authenticité2'] =  np.round((proba_authenticite[:, 1])*100,2)
df_test2

,diagonal,height_left,height_right,margin_low,margin_up,length,prédiction2,Proba authenticité2
id,,,,,,,,
B_1,172.09,103.95,103.73,4.39,3.09,113.19,True,97.28
B_2,171.52,104.17,104.03,5.27,3.16,111.82,False,2.39
B_3,171.78,103.80,103.75,3.81,3.24,113.39,True,99.64
B_4,172.02,104.08,103.99,5.57,3.30,111.10,False,0.07
B_5,171.79,104.34,104.37,5.00,3.07,111.87,False,11.04


In [233]:
df_resultat = pd.concat((df_test1[["prédiction1",'Proba authenticité1']], df_test2[["prédiction2","Proba authenticité2"]]), axis=1)

In [234]:
for id in df_resultat.index:
    if df_resultat.loc[id, 'prédiction1'] == df_resultat.loc[id, 'prédiction2']:
        df_resultat.loc[id, 'Résultat'] = df_resultat.loc[id, 'prédiction1']
    else:
        df_resultat.loc[id, 'Résultat'] = 'Billet douteux'

for id in df_resultat.index:
    if df_resultat.loc[id, 'prédiction1'] == df_resultat.loc[id, 'prédiction2']:
        df_resultat.loc[id, 'Résultat'] = df_resultat.loc[id, 'prédiction1']
        df_resultat.loc[id, 'Proba_authenticité_moyen'] = (df_resultat.loc[id, 'Proba authenticité1'] + df_resultat.loc[id, 'Proba authenticité2']) / 2
    else:
        df_resultat.loc[id, 'Résultat'] = 'Billet douteux'

for id in df_resultat.index:
   if df_resultat.loc[id, 'Résultat']==True:
          df_resultat.loc[id, 'Résultat'] = 'Vrai billet'

   elif df_resultat.loc[id, 'Résultat']==False:
        df_resultat.loc[id, 'Résultat'] = 'Faux billet'

df_resultat.drop(columns=['prédiction1','prédiction2','Proba authenticité1','Proba authenticité2'], inplace=True)
df_resultat

,Résultat,Proba_authenticité_moyen
id,,
B_1,Vrai billet,98.640
B_2,Faux billet,1.195
B_3,Vrai billet,99.820
B_4,Faux billet,0.035
B_5,Faux billet,8.520
